

In this section, we demonstrate the generation of single-cell–level regulatory graphs required for running scReGAT on matched multi-omics datasets.

As an illustrative example, we employ human pancreatic tissue data, originally reported by Wang et al.


In [1]:
import os
import sys
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
from scregat.data_process import prepare_model_input, sum_counts, plot_edge, ATACGraphDataset
import scanpy as sc
import itertools
import torch

/opt/conda/envs/scregat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/scregat/lib/python3.10/site-packages/anndata/_metadata.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import scregat
import os

print(scregat.data_process.__file__)
print(os.path.dirname(scregat.__file__))

/opt/conda/envs/scregat/lib/python3.10/site-packages/scregat/data_process.py
/opt/conda/envs/scregat/lib/python3.10/site-packages/scregat


In [3]:
ATAC_h5ad_file = "../reproduce_data/C0027_atac_celltype.h5ad"
RNA_h5ad_file = "../reproduce_data/C0027_rna_celltype.h5ad"


 - It is important to note that both **adata_rna** and **adata_atac** must contain a `celltype` column in their `obs`.  
 - The cell types present in **adata_atac** must also be represented in the `obs` of **adata_rna**.  
 - The `celltype` entries in **adata_rna** are allowed to include a larger number of categories.  


In [4]:
adata_atac = sc.read_h5ad(ATAC_h5ad_file)
adata_rna = sc.read_h5ad(RNA_h5ad_file)

In [5]:
adata_atac

AnnData object with n_obs × n_vars = 6156 × 89233
    obs: 'donor', 'celltype', 'celltype_rna'
    var: '0', '1', '2', '3', '4', '5'

In [6]:
adata_atac.obs['celltype_rna'] == adata_atac.obs['celltype']
adata_atac.obs

,donor,celltype,celltype_rna
C0027_AAACAGCCACACTAAT-1,C0027,Beta,Beta
C0027_AAACAGCCATCATGGC-1,C0027,Beta,Beta
C0027_AAACAGCCATGAGTTT-1,C0027,Alpha,Alpha
C0027_AAACATGCACTGGCTG-1,C0027,Beta,Beta
C0027_AAACATGCAGGACACA-1,C0027,Beta,Beta
...,...,...,...
C0027_TTTGTTGGTGCTCCGT-1,C0027,Beta,Beta
C0027_TTTGTTGGTGTGTCCC-1,C0027,Beta,Beta
C0027_TTTGTTGGTTAGTGAT-1,C0027,Acinar,Acinar
C0027_TTTGTTGGTTTCGCGC-1,C0027,Beta,Beta


In [7]:
adata_atac.var

,0,1,2,3,4,5
chr1-9816-10734,chr1:9816-10734,chr1:9816-10734,Peaks,chr1,9816,10734
chr1-14483-15389,chr1:14483-15389,chr1:14483-15389,Peaks,chr1,14483,15389
chr1-17065-17950,chr1:17065-17950,chr1:17065-17950,Peaks,chr1,17065,17950
chr1-28812-29619,chr1:28812-29619,chr1:28812-29619,Peaks,chr1,28812,29619
chr1-235307-236151,chr1:235307-236151,chr1:235307-236151,Peaks,chr1,235307,236151
...,...,...,...,...,...,...
GL000192.1-510831-511698,GL000192.1:510831-511698,GL000192.1:510831-511698,Peaks,GL000192.1,510831,511698
GL000192.1-512746-513650,GL000192.1:512746-513650,GL000192.1:512746-513650,Peaks,GL000192.1,512746,513650
GL000192.1-538992-539887,GL000192.1:538992-539887,GL000192.1:538992-539887,Peaks,GL000192.1,538992,539887
GL000192.1-540481-541352,GL000192.1:540481-541352,GL000192.1:540481-541352,Peaks,GL000192.1,540481,541352


In [8]:
adata_rna

AnnData object with n_obs × n_vars = 6156 × 60658
    obs: 'donor', 'celltype', 'celltype_rna'
    var: '0', '1', '2', '3', '4', '5'

In [9]:
adata_rna.obs

,donor,celltype,celltype_rna
C0027_AAACAGCCACACTAAT-1,C0027,Beta,Beta
C0027_AAACAGCCATCATGGC-1,C0027,Beta,Beta
C0027_AAACAGCCATGAGTTT-1,C0027,Alpha,Alpha
C0027_AAACATGCACTGGCTG-1,C0027,Beta,Beta
C0027_AAACATGCAGGACACA-1,C0027,Beta,Beta
...,...,...,...
C0027_TTTGTTGGTGCTCCGT-1,C0027,Beta,Beta
C0027_TTTGTTGGTGTGTCCC-1,C0027,Beta,Beta
C0027_TTTGTTGGTTAGTGAT-1,C0027,Acinar,Acinar
C0027_TTTGTTGGTTTCGCGC-1,C0027,Beta,Beta


In [10]:
adata_rna.var

,0,1,2,3,4,5
DDX11L1,ENSG00000223972.5_2,DDX11L1,Gene Expression,chr1,11868,12010
WASH7P,ENSG00000227232.5_2,WASH7P,Gene Expression,chr1,29569,29570
RP11-34P13.3,ENSG00000243485.5_4,RP11-34P13.3,Gene Expression,chr1,29553,30267
FAM138A,ENSG00000237613.2_2,FAM138A,Gene Expression,chr1,36080,36081
OR4G4P,ENSG00000268020.3_4,OR4G4P,Gene Expression,chr1,52472,52473
...,...,...,...,...,...,...
CH507-145C22.3,ENSG00000278878.1_4,CH507-145C22.3,Gene Expression,GL000195.1,137994,137995
CH17-351M24.1,ENSG00000263278.2_4,CH17-351M24.1,Gene Expression,GL000204.1,49838,49839
CH507-513H4.4,ENSG00000280614.1_4,CH507-513H4.4,Gene Expression,GL000220.1,154724,154725
CH507-513H4.3,ENSG00000281181.1_4,CH507-513H4.3,Gene Expression,GL000220.1,154724,154725


In [11]:
adata_atac.obs.celltype.unique()

['Beta', 'Alpha', 'Delta', 'Acinar', 'Immune', 'Stellate', 'Ductal', 'Gamma']
Categories (8, object): ['Acinar', 'Alpha', 'Beta', 'Delta', 'Ductal', 'Gamma', 'Immune', 'Stellate']

In [12]:
adata_rna.obs.celltype.unique()

['Beta', 'Alpha', 'Delta', 'Acinar', 'Immune', 'Stellate', 'Ductal', 'Gamma']
Categories (8, object): ['Acinar', 'Alpha', 'Beta', 'Delta', 'Ductal', 'Gamma', 'Immune', 'Stellate']

# Check whether the cell type order matches

In [13]:
adata_rna.obs_names == adata_atac.obs_names

array([ True,  True,  True, ...,  True,  True,  True])

In [14]:
adata_rna.var_names_make_unique()
adata_atac.var_names_make_unique()

In [15]:
adata_rna.obs['celltype'] = adata_rna.obs['celltype'].astype('object')
df_rna = sum_counts(adata_rna,by = 'celltype', marker_gene_num=300)

**finished identifying marker genes by COSG**


In [16]:
df_rna

,UNC5B,NEDD4L,DHRS3,CHKA,KLF6,GPR160,YIPF1,HDAC4,TJP1,EPB41L2,...,INPP4B,TUBA1C,FRMD8,HEG1,SVOP,NOTCH2,ASL,ZBTB20,DUSP22,RP11-319G6.1
celltype,,,,,,,,,,,,,,,,,,,,,
Acinar,33,2706,196,437,557,163,38,120,1389,307,...,666,15,18,40,9,110,83,5001,149,108
Alpha,15,1630,30,1739,161,129,157,1611,2874,354,...,1188,82,104,60,754,26,106,8245,97,1251
Beta,104,3509,34,1149,756,35,305,1769,3864,1059,...,4899,222,116,94,943,58,156,32099,152,4275
Delta,97,428,3,307,45,50,57,281,609,234,...,736,5,26,12,322,12,23,4013,29,569
Ductal,8,214,7,52,59,9,3,15,186,31,...,49,8,4,3,5,22,9,578,6,22
Gamma,1,10,0,17,7,4,6,12,34,32,...,120,3,1,2,17,0,2,208,0,14
Immune,19,25,0,24,27,0,1,23,55,53,...,100,2,2,40,0,34,2,309,2,18
Stellate,19,33,12,58,46,7,4,22,37,70,...,15,2,10,7,3,25,6,183,8,16


In [17]:
import scregat
import os
# This step adds tissue-specific Hi-C regulatory relationships.
# The user needs to provide a set of files. For example,
#in 
base_dir = '../data/'
os.listdir(base_dir)
# we include a file called PO_brain.txt,
# which contains brain tissue-specific Hi-C links.


['PO.txt',
 'all_tissue_SNP_Gene.txt',
 'TF_Gene_tissue_cutoff1.csv',
 'PO_brain.txt',
 'genes.protein.tss.tsv',
 'hg38.chrom.sizes',
 'PP.txt',
 'readme.md',
 'trrust_rawdata.human.tsv',
 'celltype_specific_cRE_interactions',
 'PP_brain.txt',
 'TF_Gene_tissue_Brain.csv',
 'model_init.pth',
 'dataset_atac_kRG_Pancreas.pkl',
 'processed_files',
 'peaks_process']

In [18]:
adata_atac

AnnData object with n_obs × n_vars = 6156 × 89233
    obs: 'donor', 'celltype', 'celltype_rna'
    var: '0', '1', '2', '3', '4', '5'

In [20]:

# Note that regardless of whether the multi-omics data are matched at the single-cell level, 
# the regulatory graph will be constructed under the unmatched configuration. 
# Single-cell–level matched features will instead be incorporated during model training.


dataset_obj = prepare_model_input(
    # [Core Data] Single-cell ATAC-seq AnnData object.
    # Requirement: .X must be Peak-by-Cell matrix; .obs must contain cell type annotations.
    adata_atac = adata_atac,
    
    # [Output Path] Root directory for storing intermediate processed files.
    # The script will create a 'processed_files' folder here (e.g., sorted bed files).
    path_data_root = '../data',
    
    # [File Reference] String path to the original ATAC file (used for naming/logging).
    file_atac = ATAC_h5ad_file, 
    
    # [Core Data] RNA expression matrix aggregated by cell type (Pseudo-bulk).
    # Format: Pandas DataFrame. 
    # Index (Rows): Cell type names (must match 'celltype_rna' in adata_atac.obs).
    # Columns: Gene Symbols (e.g., 'TP53').
    df_rna_celltype = df_rna,
    
    # [Prior Knowledge] Path to eQTL (expression Quantitative Trait Loci) data.
    # Used to link SNPs/non-coding regions to target genes.
    path_eqtl = '../data/all_tissue_SNP_Gene.txt',
    
    # [Prior Knowledge] Suffix for Hi-C interaction files to specify tissue context.
    # e.g., if set to "_brain", the code looks for 'PP_brain.txt' and 'PO_brain.txt'.
    # PP = Promoter-Promoter, PO = Promoter-Other (Enhancer).
    # Assuming that tissue-specific Hi-C data are not available, this option may be omitted. 
    Hi_C_file_suffix = "",  
    # [Preprocessing] Whether to convert genomic coordinates from hg19 to hg38.
    # Set True if input ATAC peaks are hg19 (requires LiftOver tool); False if already hg38.
    hg19tohg38 = True,
    data_dir = '../data',
    liftover_path = '../liftOver',
    chain_file = '../hg19ToHg38.over.chain.gz',
    
    # [QC Filter] Peak filtering threshold.
    # Peaks must be accessible in at least 1% (0.01) of cells to be retained.
    min_percent = 0.01,
    
    # [Prior Knowledge] Whether to use an extended TF (Transcription Factor) database.
    # False: Uses TRRUST only (Curated, high-confidence).
    # True: Uses TRRUST + CHEA3/ChIP-seq aggregated data.
    use_additional_tf = True,
    
    # [Prior Knowledge] Reliability threshold for the extended TF database.
    # Only applies if use_additional_tf=True.
    # Integer indicating in how many tissues/datasets the TF-Gene link must appear to be kept.
    # 10 indicates a high-confidence, conserved regulatory relationship.
    tissue_cuttof = 10
)

only dataset_obj ...
执行 liftover 命令: "../liftOver" "../data/processed_files/peaks.sort.bed" "../hg19ToHg38.over.chain.gz" "../data/peaks_process/peaks_hg38.bed" "../data/peaks_process/unmap.bed"


Reading liftover chains
Mapping coordinates


processing Hi-C ...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


processing TF ...
additional TF...
total candidate tf-gene:  28054


In [21]:
dataset_obj.list_graph[0]

Data(x=[29826, 1], edge_index=[2, 34203], y=[1], edge_tf=[359, 2], y_exp=[1634], cell='C0027_AAACAGCCACACTAAT-1')

In [22]:
file_atac_test = '../data/dataset_atac_kRG_Pancreas.pkl'
with open(file_atac_test, 'wb') as w_pkl:
    str_pkl = pickle.dumps(dataset_obj)
    w_pkl.write(str_pkl)



## Training the Model and Obtaining Regulatory Scores

You can train the model and generate regulatory scores directly from the command line.  
For example, we used the following command to produce a sample dataset and a model weight file.  

**Note:** In this example we explicitly enabled single‑cell–level matched features by using the `--use_sc_exp` flag together with the `--rna_file` argument:

```bash
python ./scReGAT/run_scregat_cli.py \
  --input_file ./dataset_atac_kRG_Pancreas.pkl \
  --output_file ./RS_score_Pancreas.h5ad \
  --use_sc_exp \
  --rna_file ./C0027_rna_celltype.h5ad \
  --save_model_path ./Pancreas.pth \
  --load_model_path ./scReGAT/data/model_init.pth \
  --gpu 2
```

---

## Argument Description

```python
def parse_args():
    parser = argparse.ArgumentParser(description="Run scReGAT Model Training and Inference")

    # --- I/O parameters ---
    parser.add_argument('--input_file', type=str, required=True, 
                        help='Path to the input ATAC pickle file (e.g., dataset_atac_core_MFG.pkl)')
    parser.add_argument('--output_file', type=str, required=True, 
                        help='Path to save the output AnnData file (e.g., result.h5ad)')

    # --- Single-cell expression integration parameters ---
    parser.add_argument('--use_sc_exp', action='store_true',
                        help='Enable integration of single-cell RNA expression data into graph node features')
    parser.add_argument('--rna_file', type=str, default=None,
                        help='Path to the RNA .h5ad file; required if --use_sc_exp is set')

    # --- Model saving and loading ---
    parser.add_argument('--save_model_path', type=str, default=None, 
                        help='Optional path to save trained model parameters')
    parser.add_argument('--load_model_path', type=str, default=None, 
                        help='Optional path to load pre-trained model parameters')

    # --- Training control ---
    parser.add_argument('--skip_train', action='store_true',
                        help='Skip the training phase and run inference directly')
    parser.add_argument('--seed', type=int, default=1233, 
                        help='Random seed (default: 1233)')
    parser.add_argument('--epochs', type=int, default=4, 
                        help='Number of training epochs (default: 4)')
    parser.add_argument('--lr', type=float, default=1e-4, 
                        help='Learning rate (default: 1e-4)')
    parser.add_argument('--batch_size', type=int, default=15, 
                        help='Training batch size (default: 15)')
    parser.add_argument('--sparse_loss_weight', type=float, default=0.1, 
                        help='Weight for sparse loss (default: 0.1)')

    # --- Testing / inference parameters ---
    parser.add_argument('--test_batch_size', type=int, default=20, 
                        help='Batch size for inference (default: 20)')
    parser.add_argument('--test_ratio', type=float, default=0.5, 
                        help='Ratio of cells to use for testing (default: 0.5)')

    # --- Hardware parameters ---
    parser.add_argument('--gpu', type=int, default=1, 
                        help='GPU ID to use; set -1 to use CPU (default: 1)')

    return parser.parse_args()
```

---

## Recommendation

We recommend users initialize training on new datasets using the model weight file:

```
./scReGAT/data/model_init.pth
```

This provides a stable starting point for training and helps ensure stability across experiments.  

---
